<a href="https://colab.research.google.com/github/e-romario/assistente-ai-trabalhos-academicos/blob/main/Assistente_de_Pesquisa_Acad%C3%AAmica_com_IA_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
# --- Configurações Iniciais e Instalação de Bibliotecas ---
%pip -q install google-genai
%pip -q install pillow
%pip -q install requests

In [102]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [103]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

# Essas variáveis ​​devem ser definidas antes das funções que as utilizam como argumentos padrão
MODEL_ID = "gemini-2.0-flash"
MODEL_PRO = "gemini-2.0-flash" # Usar o PRO para agentes que exigem mais capacidade

In [104]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Qual é a data de hoje?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Hoje é quinta-feira, 2 de maio de 2024.


In [105]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Qual é a data de hoje?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A data de hoje é 17 de maio de 2025.

In [106]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['data de hoje']
Páginas utilizadas na resposta: wincalendar.com, wikidates.org, wikidates.org



In [107]:
# Instalar Framework de agentes do Google ################################################
%pip -q install google-genai google-adk

In [108]:
# Principais importações
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings("ignore")
from docx import Document
from io import BytesIO
from google.colab import files

In [109]:
# Função auxiliar para enviar uma mensagem para um agente via Runner
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
            final_response += "\n"
    return final_response

In [110]:
# Função auxiliar para exibir texto formatado em Markdown
def to_markdown(text):
    text = text.replace('•', ' *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [111]:
##################################################
# --- Agente 1: Busca Bibliográfica ---
##################################################
def agente_busca_bibliografica(topico):
    buscador = Agent(
        name="agente_busca_bibliografica",
        model=MODEL_ID,
        instruction="""
        Você é um assistente de pesquisa bibliográfica. Sua tarefa é usar a ferramenta de busca do Google (google_search)
        para encontrar artigos científicos relevantes sobre o tópico fornecido. Foque em identificar os títulos e os links
        dos artigos mais relevantes. Tente encontrar até 5 artigos e retorne o título e o link de cada um. Siga estritamente o formato:
        Título: [Título do Artigo]
        Link: [Link do Artigo]

        Não adicione texto extra antes ou depois de cada par Título/Link.
        """,
        description="Agente que busca artigos científicos e seus links no Google",
        tools=[google_search]
    )
    entrada_busca = f"Tópico de pesquisa: {topico}"
    resultados = call_agent(buscador, entrada_busca)

    # Parse the results string to extract titles and links
    links_encontrados = []
    lines = resultados.strip().split('\n')
    titulo = None
    link = None
    for line in lines:
        line = line.strip()
        if line.startswith("Título:"):
            titulo = line.replace("Título:", "").strip()
        elif line.startswith("Link:") or line.startswith("URL:"):
            link = line.replace("Link:", "").replace("URL:", "").strip()
            if titulo and link:
                links_encontrados.append((titulo, link))
                titulo = None  # Reset for the next article
                link = None    # Reset for the next article

    return resultados, links_encontrados

In [112]:
##################################################
# --- Agente 2: Análise de Artigos ---
##################################################
def agente_analise_artigos(topico, resultados_busca):
    analisador = Agent(
        name="agente_analise_artigos",
        model=MODEL_PRO,
        instruction=f"""
        Você é um analista de artigos científicos. Dada a lista de artigos encontrados sobre o tópico "{topico}",
        sua tarefa é tentar extrair as informações chave de cada um. Se possível, identifique o objetivo, a metodologia,
        os principais resultados e as conclusões de cada artigo listado. Use a busca do Google se precisar de mais detalhes
        sobre algum artigo específico.

        Lista de artigos:
        {resultados_busca}
        """,
        description="Agente que analisa artigos científicos e extrai informações chave",
        tools=[google_search]
    )
    entrada_analise = f"Tópico: {topico}\nResultados da busca: {resultados_busca}"
    analise = call_agent(analisador, entrada_analise)
    return analise

In [113]:
##################################################
# --- Agente 3: Síntese de Literatura ---
##################################################
def agente_sintese_literatura(topico, analise_artigos):
    sintetizador = Agent(
        name="agente_sintese_literatura",
        model=MODEL_PRO,
        instruction=f"""
        Você é um sintetizador de literatura acadêmica. Com base na análise dos artigos sobre o tópico "{topico}",
        sua tarefa é agrupar e resumir as principais descobertas, identificar temas comuns, divergências e possíveis
        lacunas na pesquisa. Crie um resumo conciso da literatura relevante.
        """,
        description="Agente que sintetiza a literatura acadêmica",
    )
    entrada_sintese = f"Tópico: {topico}\nAnálise dos artigos: {analise_artigos}"
    sintese = call_agent(sintetizador, entrada_sintese)
    return sintese

In [114]:
##################################################
# --- Agente 4: Agente de Formatação ---
##################################################
def agente_formatacao(sintese_literatura, formato_usuario=None, arquivo_modelo=None):
    formatador = Agent(
        name="agente_formatacao",
        model=MODEL_PRO,
        instruction="""
        Você é um especialista em formatação de documentos acadêmicos. Sua tarefa é formatar o texto fornecido
        de acordo com as instruções do usuário ou usando um modelo fornecido. Se nenhum formato ou modelo for especificado,
        organize as informações de forma lógica e coerente.
        """,
        description="Agente que formata documentos acadêmicos",
    )
    entrada_formatacao = f"Síntese da literatura:\n{sintese_literatura}\n\nFormato desejado: {formato_usuario}\nArquivo modelo (se fornecido): {arquivo_modelo is not None}"
    texto_formatado = call_agent(formatador, entrada_formatacao)
    return texto_formatado

In [115]:
##################################################
# --- Agente 5: Agente de Saída para o Usuário ---
##################################################
def agente_saida_usuario(texto_formatado, links_bibliograficos):
    print("\n--- 📝 Resultado do Agente de Saída (Visualização) ---\n")
    display(Markdown("> Este é um resumo estruturado da Pesquisa:\n"))
    display(Markdown(textwrap.indent(texto_formatado, '> ')))

    if links_bibliograficos:
        print("\n--- 🔗 Links de Pesquisa/Referências Bibliográficas ---\n")
        for titulo, link in links_bibliograficos:
            display(Markdown(f"* **{titulo}:** {link}"))

    return "Visualização do arquivo e links exibidos."

In [116]:
##################################################
# --- Fluxo Principal ---
##################################################
if __name__ == "__main__":
    print("📚 Iniciando o Assistente de Pesquisa Acadêmica 📚")
    topico_pesquisa = input("❓ Por favor, digite o TÓPICO da sua pesquisa: ")

    if not topico_pesquisa:
        print("Você esqueceu de digitar o tópico da pesquisa!")
    else:
        print(f"\n🔍 Buscando artigos sobre '{topico_pesquisa}'...")
        resultados_busca, links_encontrados = agente_busca_bibliografica(topico_pesquisa)
        print("\n--- 📝 Resultado do Agente de Busca Bibliográfica ---\n")
        display(to_markdown(resultados_busca))
        print("--------------------------------------------------------------")

        print(f"\n🧐 Analisando os artigos sobre '{topico_pesquisa}'...")
        analise_artigos = agente_analise_artigos(topico_pesquisa, resultados_busca)
        print("\n--- 📝 Resultado do Agente de Análise de Artigos ---\n")
        display(to_markdown(analise_artigos))
        print("--------------------------------------------------------------")

        print(f"\n✍️ Sintetizando a literatura sobre '{topico_pesquisa}'...")
        sintese_literatura = agente_sintese_literatura(topico_pesquisa, analise_artigos)
        print("\n--- 📝 Resultado do Agente de Síntese de Literatura ---\n")
        display(to_markdown(sintese_literatura))
        print("--------------------------------------------------------------")

        formato_desejado = input("\n❓ Deseja especificar um formato (exemplo: monografia, artigo, entrevista, etc)? Se sim, digite-o. Deixe em branco para formatação padrão: ")

        print("\n🎨 Formatando a síntese...")
        texto_formatado = agente_formatacao(sintese_literatura, formato_usuario=formato_desejado)
        print("\n--- 📝 Resultado do Agente de Formatação ---\n")
        display(to_markdown(texto_formatado))
        print("--------------------------------------------------------------")

        agente_saida_usuario(texto_formatado, links_encontrados)

print("\n✨ Processo concluído!")

📚 Iniciando o Assistente de Pesquisa Acadêmica 📚


KeyboardInterrupt: Interrupted by user